1. How do you add a path that didn't exist to sampling missing values.
    e.g. Falls example: We know that some people who were assessed and high risk were not referred or treated. By design was allowed, but was not observed. How do you add this path?
        And show how you add multiple paths.

2. Change NaNs to something else. "miss" or something. Initiating the class. Add label for structural and sampling missing values.

